<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO _______________________________________________________________________________

# Solicição ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação os nomes dos bairros e inserir na lista
for i in range(n_bairros):  # repetição considerando a quantidade informada pelo usuário
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)

# Solicitação ao usuário com relação ao número de habitantes de cada bairro e barrar
#valor inválido
for bairro in nomes_bairros:
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break
# ________________________________________________________________________________

print('=-=' * 30)
print()
print('::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::')
print()
print('=-=' * 30)
print()

#_____________________________________________________________________________________
# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA E ARAMZENAMENTO

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal=[]

for i, bairro in enumerate(nomes_bairros):
  # Total de habitantes no bairro atual
  n_hab = n_hab_bairro[i]

    # Quantidade de resíduos gerados semanalmente por um habitante
  residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Calculo da quantidade de resíduos para o bairro
  residuos_bairro_semanal = round(n_hab * residuos_por_habitante_semanal)

  residuos_por_bairro_semanal.append(residuos_bairro_semanal)

 # Volume necessário por m³ para armazenar os resíduos nos depósitos:

  volumes_necess_metrocub = []
  peso_por_metro_cubico_necessario = 631 # quantidade de kg que corresponde a 1 m³
  for i, residuos_bairro_semanal in enumerate(residuos_por_bairro_semanal):
    vol_necess_metrocub = residuos_bairro_semanal / peso_por_metro_cubico_necessario
    volumes_necess_metrocub.append(vol_necess_metrocub) # volume necessario no deposito para os resíduos

  # PARTE 3 - DIMENSIONAMENTOS _________________________________________________________________________________________________________________

  # Dimensões da estação de coleta:

  larg_estacao = 20 # metros
  compri_estacao = 30 # metros
  altura_estacao = 7 # metros

  # Dimensões de cada depósito:

  larg_dep = 10 # metros
  compri_dep = 20 # metros
  altura_dep = 7 # metros

  # Dimenções da área de triagem:

  larg_triagem = 20 # metros
  compri_triagem = 10 # metros
  altura_triagem = 7 # metros

  #_____________________________________________________________________________________________

  # Definição da quantidade de estações

  # Listas - quantidadae de estações / quantidade de caminhões
  quantidades_estacoes_por_bairro = []
  quantidades_caminhao_por_bairro = []

  rs_ocupa = residuos_por_bairro_semanal / peso_por_metro_cubico_necessario
  quatidade_estacao = rs_ocupa / vol_necess_metrocub



  quantidades_estacoes_por_bairro.append(quantidade_estacao)


  #________________________________________________________________________________________________________________________
  # PARTE 4 - NÚMERO DE CAMINHÕES PARA A(S) ESTAÇÕES DE COLETA

  # NUMERO DE CAMINHÕES DESCARREGANDO OS RESÍDUOS PRÉ TRIADOS E LEVANDO OS RESÍDUOS TRIADOS

  quant_caminhao = 0
  caminhao_kg = 12000  # Capacidade maxima em kg

  while residuos_bairro_semanal > caminhao_kg:
      residuos_rest = residuos_bairro_semanal - caminhao_kg
      quant_caminhao += 1
      residuos_bairro_semanal = residuos_rest

  # If there are remaining waste less than or equal to the truck capacity
  if residuos_bairro_semanal <= caminhao_kg and residuos_bairro_semanal > 0:
      quant_caminhao += 1

  quant_caminhao_total_bairro = quant_caminhao


  # If there are remaining waste less than or equal to the truck capacity
  if residuos_bairro_semanal <= caminhao_kg and residuos_bairro_semanal > 0:
      quant_caminhao += 1

  quant_caminhao_total_bairro = quant_caminhao  # Total number of trucks needed

  quantidades_caminhao_por_bairro.append(quant_caminhao_total_bairro)

  #____________________________________________________________________________________________________________________________

  # PARTE 5 - IMPRIMINDO RESULTADOS

  # Informações por bairro da cidade: Dimensionamento da estação, dos depósitos, do setor de triagem e a quantidade de estações de coletas:
for i, bairro in enumerate(nomes_bairros):
    print(f'Residuos semanais por bairro : {residuos_por_bairro_semanal[i]}')
    print(f'Bairro: {bairro}')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('Dimensões de cada depósito da(s) Estações(s):')
    print()
    print(f'Largura: {larguras_dep_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_dep_por_bairro[i]} m')
    print(f'Área: {areas_dep_por_bairro[i]} m²')
    print()
    print('Dimensões do Setor de Triagem:')
    print()
    print(f'Largura: {larguras_triagem_por_bairro[i]} m')
    print(f'Comprimento: 10 m')
    print(f'Área: {areas_triagem_por_bairro[i]} m²')
    print()
    print('Dimensões da Estação de coleta e Triagem:')
    print()
    print(f'Largura: {larguras_estacao_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_estacao_por_bairro[i]} m')
    print(f'Área: {areas_estacao_por_bairro[i]} m²')
    print(f'Altura: 7 m')
    print()
    print(f'Quantidade de Estações de Coleta e Triagem necessária para o bairro: { quatidade_estacao[i]} unidades')
    print()
    print('=-=' * 30)
    print()
    print('Dados importantes:')
    print()
    print(f"Total de caminhões necessarios : {quant_caminhao_total_bairro}")
    print()
    print('=-=' * 30)
    print()



Informe o nome da cidade: natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 46485
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: asas
Digite o nome do 2º bairro: casas
Digite o número de habitantes do bairro asas: 122
Digite o número de habitantes do bairro casas: 312
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=



TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [17]:
import math

# Função para calcular a quantidade de caminhões necessários
def calcular_quantidade_caminhoes(residuos_bairro_semanal):
    quant_caminhao = 0
    caminhao_kg = 12000  # Capacidade máxima em kg

    while residuos_bairro_semanal > caminhao_kg:
        residuos_rest = residuos_bairro_semanal - caminhao_kg
        quant_caminhao += 1
        residuos_bairro_semanal = residuos_rest

    if residuos_bairro_semanal > 0:
        quant_caminhao += 1

    return quant_caminhao

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO

print("=-=" * 30)
print("Bem-vindo ao dimensionamento das estações de coleta da cidade!")
print("=-=" * 30)
print()

# Solicitação ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação aos nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)

# Solicitação ao usuário com relação ao número de habitantes de cada bairro
for bairro in nomes_bairros:
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break

print()
print("=-=" * 30)
print()
print('Dimensionamento das estações de coleta a serem construídas na cidade')
print()
print("=-=" * 30)
print()

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal = []

for i, bairro in enumerate(nomes_bairros):
    # Quantidade de resíduos gerados semanalmente por um habitante
    residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Calculo da quantidade de resíduos para o bairro
    residuos_bairro_semanal = round(n_hab_bairro[i] * residuos_por_habitante_semanal)
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

    # Dimensões dos depósitos, setor de triagem e estação de coleta para o bairro
    larg_dep_max = 10
    compri_dep_max = 20

    # Volume necessário por m³ para armazenar os resíduos nos depósitos
    peso_por_metro_cubico_necessario = 631  # kg que corresponde a 1 m³
    vol_necess_metrocub = residuos_bairro_semanal / peso_por_metro_cubico_necessario
    volumes_necess_metrocub.append(vol_necess_metrocub)

    # Dimensões dos depósitos
    larg_dep = min(larg_dep_max, (insira o calculo da largura de forma que sua multiplicação com o comprimento variavel e a altura que vale 7 corresponda o tamanho em metro cubico de vol_necess_metrocub ) )
    compri_dep = min(compri_dep_max, larg_dep * 2)
    area_dep_total = larg_dep * compri_dep

    # Quantidade de estações de coleta
    quant_estacao = math.ceil(area_dep_total / (10 * 20))  # Cada estação cobre uma área de 200 m²

    # Dimensões do setor de triagem
    larg_triagem = 2 * larg_dep
    area_triagem = larg_triagem * 10

    # Dimensões da estação de coleta e triagem
    larg_estacao = 2 * larg_dep
    compri_estacao = compri_dep + 10  # 10 metros para o setor de triagem
    area_estacao = larg_estacao * compri_estacao

    # Número de funcionários necessários

    # Número de caminhões necessários
    quant_caminhao_total_bairro = calcular_quantidade_caminhoes(residuos_bairro_semanal)

    # Impressão dos resultados para cada bairro
    print(f'Residuos semanais por bairro: {residuos_bairro_semanal}')
    print(f'Bairro: {bairro}')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('Dimensões de cada depósito da(s) Estações(s):')
    print(f'Largura: {larg_dep:.2f} m')
    print(f'Comprimento: {compri_dep:.2f} m')
    print(f'Área: {area_dep_total:.2f} m²')
    print()
    print('Dimensões do Setor de Triagem:')
    print(f'Largura: {larg_triagem:.2f} m')
    print(f'Comprimento: 10 m')
    print(f'Área: {area_triagem:.2f} m²')
    print()
    print('Dimensões da Estação de coleta e Triagem:')
    print(f'Largura: {larg_estacao:.2f} m')
    print(f'Comprimento: {compri_estacao:.2f} m')
    print(f'Área: {area_estacao:.2f} m²')
    print()
    print(f'Quantidade de Estações de Coleta e Triagem necessária para o bairro: {quant_estacao} unidades')
    print()
    print('=-=' * 30)
    print()
    print('Dados importantes:')
    print()
    print(f'Total de caminhões necessários: {quant_caminhao_total_bairro}')
    print()
    print('=-=' * 30)
    print()


=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
Bem-vindo ao dimensionamento das estações de coleta da cidade!
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Informe o nome da cidade: natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 10000000
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: asas
Digite o nome do 2º bairro: casas
Digite o número de habitantes do bairro asas: 85546
Digite o número de habitantes do bairro casas: 4654

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Dimensionamento das estações de coleta a serem construídas na cidade

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Residuos semanais por bairro: 2371009
Bairro: asas
Número de habitantes: 85546

Dimensões de cada depósito da(s) Estações(s):
Largura: 10.00 m
Comprimento: 20.00 m
Área